In [1]:
import flexflow.serve as ff
import json, os
from types import SimpleNamespace
import random

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Data comes from https://huggingface.co/datasets/databricks/databricks-dolly-15k
def import_dataset(dataset_size=10, inference_percentage=0.6):
    dataset = load_dataset("databricks/databricks-dolly-15k", split="train")
    data = []
    for i,row in enumerate(dataset):
        if i == dataset_size:
            break
        if len(row['context']) == 0:
            data.append((row['instruction'],row['response']))
    inference_prompts = []
    finetuning_prompts = []
    for d in data:
        if random.random() <= inference_percentage:
            inference_prompts.append(d[0])
        else:
            finetuning_prompts.append(d)
    return inference_prompts, finetuning_prompts

In [3]:
configs_dict = {
    # required parameters
    "num_gpus": 1,
    "memory_per_gpu": 8192,
    "zero_copy_memory_per_node": 12000,
    # optional parameters
    "num_cpus": 4,
    "legion_utility_processors": 4,
    "data_parallelism_degree": 1,
    "tensor_parallelism_degree": 1,
    "pipeline_parallelism_degree": 1,
    "offload": False,
    "offload_reserve_space_size": 8 * 1024,  # 8GB
    "use_4bit_quantization": False,
    "use_8bit_quantization": False,
    "enable_peft": True,
    "peft_activation_reserve_space_size": 1024,  # 1GB
    "peft_weight_reserve_space_size": 1024,  # 1GB
    "profiling": False,
    "inference_debugging": False,
    "fusion": False,
}
model_configs = {
    # required parameters
    "base_model": "JackFram/llama-160m",
    "inference_peft_model_id": "goliaro/llama-160m-lora",
    "finetuning_peft_model_id": "goliaro/llama-160m-lora",
    # optional parameters
    "cache_path": "",
    "refresh_cache": False,
    "full_precision": True,
    # relative paths
    "prompt": "",
    "finetuning_dataset": "../../rdelacou/peft_dataset.json",
    "output_file": "../output/ff_peft.txt",
}
# Merge dictionaries
configs_dict.update(model_configs)
configs = SimpleNamespace(**configs_dict)

In [4]:
# Initialize the FlexFlow runtime. ff.init() takes a dictionary or the path to a JSON file with the configs
def init_llm(configs_dict, configs):
    ff.init(configs_dict)

    # Create the FlexFlow LLM
    ff_data_type = (
        ff.DataType.DT_FLOAT if configs.full_precision else ff.DataType.DT_HALF
    )
    llm = ff.LLM(
        configs.base_model,
        data_type=ff_data_type,
        cache_path=configs.cache_path,
        refresh_cache=configs.refresh_cache,
        output_file=configs.output_file,
    )
    # Add inference and/or finetuning lora
    lora_inference_config = None
    lora_finetuning_config = None
    if len(configs.prompt) > 0:
        lora_inference_config = ff.LoraLinearConfig(
            llm.cache_path, configs.inference_peft_model_id
        )
        llm.add_peft(lora_inference_config)
    if len(configs.finetuning_dataset) > 0:
        # lora_finetuning_config = ff.LoraLinearConfig(
        #     llm.cache_path,
        #     configs.finetuning_peft_model_id,
        #     target_modules=["down_proj"],
        #     rank=16,
        #     lora_alpha=16,
        #     trainable=True,
        #     init_lora_weights=True,
        #     optimizer_type=ff.OptimizerType.OPTIMIZER_TYPE_SGD,
        # )
        lora_finetuning_config = ff.LoraLinearConfig(
            llm.cache_path,
            configs.inference_peft_model_id,
            trainable=True,
            optimizer_type=ff.OptimizerType.OPTIMIZER_TYPE_SGD,
            optimizer_kwargs={
                "learning_rate": 1.0,
                "momentum": 0.0,
                "weight_decay": 0.0,
                "nesterov": False,
            },
        )
        llm.add_peft(lora_finetuning_config)

    # Compile the LLM for inference and load the weights into memory
    generation_config = ff.GenerationConfig(
        do_sample=False, temperature=0.9, topp=0.8, topk=1
    )
    llm.compile(
        generation_config,
        enable_peft_finetuning=(len(configs.finetuning_dataset) > 0),
        max_requests_per_batch=1,
        max_seq_length=256,
        max_tokens_per_batch=128,
    )
    return llm, lora_inference_config, lora_finetuning_config

In [5]:
llm, lora_inference_config, lora_finetuning_config = init_llm(configs_dict, configs)

llm.start_server()

requests = []
# Serving
if len(configs.prompt) > 0:
    prompts = [s for s in json.load(open(configs.prompt))]
    inference_requests = [
        ff.Request(
            ff.RequestType.REQ_INFERENCE,
            prompt=prompt,
            max_sequence_length=128,
            peft_model_id=llm.get_ff_peft_id(lora_inference_config),
        )
        for prompt in prompts
    ]
    requests += inference_requests
# Finetuning
if len(configs.finetuning_dataset) > 0:
    finetuning_request = ff.Request(
        ff.RequestType.REQ_FINETUNING,
        max_sequence_length=128,
        peft_model_id=llm.get_ff_peft_id(lora_finetuning_config),
        dataset_filepath=os.path.join(os.getcwd(), configs.finetuning_dataset),
        max_training_steps=2,
    )
    requests.append(finetuning_request)

llm.generate(requests)

llm.stop_server()

[0 - 7f9b6a648280]    0.256053 {3}{Mapper}: Enabled Control Replication Optimizations.
[0 - 7f9b6a648280]    0.256113 {3}{Mapper}: Enabled Control Replication Optimizations.
[0 - 7f9b6a648280]    0.256122 {3}{Mapper}: Enabled Control Replication Optimizations.
[0 - 7f9b6a648280]    0.256129 {3}{Mapper}: Enabled Control Replication Optimizations.
[0 - 7f9b6a648280]    0.256135 {3}{Mapper}: Enabled Control Replication Optimizations.
/opt/conda/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
workSpaceSize (128 MB)


: 